##BioMistral Medical RAG Chatbot Using BioMistral Open Source LLM
In the notebook we built a medical Chat bot with biomistral LLM and Heart Heath pdf file

##load the google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


##Installation

In [4]:
pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

##Importing libraries

In [5]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

##Import the document

In [6]:
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/Heart_health/data")
docs = loader.load()

In [ ]:
len(docs)


95

In [7]:
docs[6]

Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'title': 'Your Guide to A Healthy Heart', 'source': '/content/drive/MyDrive/Heart_health/data/healthyheart.pdf', 'total_pages': 95, 'page': 6, 'page_label': '7'}, page_content='2\nThese facts may seem frightening, but they need not be. The good\nnews is that you have a lot of power to protect and improve your\nheart health. This guidebook will help you find out your own risk\nof heart disease and take steps to prevent it.\n“But,” you may still be thinking, “I take pretty good care of myself.\nI’m unlikely to get heart disease.” Yet a recent national survey shows\nthat only 3 percent of U.S. adults practice all of the “Big Four”\nhabits tha

##Chunking

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

In [ ]:
len(chunks)


585

In [ ]:
chunks[90]


Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'title': 'Your Guide to A Healthy Heart', 'source': '/content/drive/MyDrive/Heart_health/data/healthyheart.pdf', 'total_pages': 95, 'page': 14, 'page_label': '15'}, page_content='been told by your doctor that your blood pressure is too high?\n● Has your doctor told you that your LDL “bad” cholesterol is \ntoo high; that your total cholesterol level is 200 mg/dL or higher;\nOR, that your HDL “good” cholesterol is less than 40 mg/dL?')

##Embedding Creations

In [ ]:
import os
os.environ['HUGGINGACEHUB_API_TOKEN']=""

In [10]:
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

/tmp/ipython-input-45285844.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

##Vector Store Creation

In [11]:
vectorstore= Chroma.from_documents(chunks, embeddings)

In [12]:
query='who is at risk of heart attack'
search_results = vectorstore.similarity_search(query)

In [13]:
search_results

[Document(metadata={'title': 'Your Guide to A Healthy Heart', 'page': 14, 'page_label': '15', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'author': 'NHLBI', 'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'moddate': '2006-02-23T09:58:15-05:00', 'subject': 'Heart disease', 'creationdate': '2006-02-16T11:30:29-05:00', 'creator': 'QuarkXPress(tm) 6.5', 'total_pages': 95, 'source': '/content/drive/MyDrive/Heart_health/data/healthyheart.pdf'}, page_content='on most days?\n● Has a doctor told you that you have angina (chest pains); \nOR, have you had a heart attack?\nIf you answered “yes” to any of these\nquestions, you have a higher risk of\nhaving a heart attack. Read on to\nlearn what you can do to lower\nyour risk. \nRating Your\nRisk'),
 Document(metadata={'source': '/content/drive/MyDrive/Heart_health/data/healthyheart.pdf', 'page': 25, 'author': 'NHLBI', 'creator': 'QuarkXPress(tm) 6.5', 'total_pages': 95, 'cr

In [14]:
retriver=vectorstore.as_retriever(search_kwargs={'k':5})

In [15]:
retriver.get_relevant_documents(query)

/tmp/ipython-input-4018303955.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriver.get_relevant_documents(query)


[Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creationdate': '2006-02-16T11:30:29-05:00', 'author': 'NHLBI', 'moddate': '2006-02-23T09:58:15-05:00', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'title': 'Your Guide to A Healthy Heart', 'creator': 'QuarkXPress(tm) 6.5', 'page_label': '15', 'source': '/content/drive/MyDrive/Heart_health/data/healthyheart.pdf', 'subject': 'Heart disease', 'total_pages': 95, 'page': 14}, page_content='on most days?\n● Has a doctor told you that you have angina (chest pains); \nOR, have you had a heart attack?\nIf you answered “yes” to any of these\nquestions, you have a higher risk of\nhaving a heart attack. Read on to\nlearn what you can do to lower\nyour risk. \nRating Your\nRisk'),
 Document(metadata={'moddate': '2006-02-23T09:58:15-05:00', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'cr

##LLM Model loading

In [16]:
llm=LlamaCpp(
    model_path="/content/drive/MyDrive/Heart_health/BioMistral-7B.Q4_K_M.gguf",
              temperature=0.2,
              max_tokens=2048,
              top_p=1,
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/Heart_health/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.a

## Use LLM and retriver and Query to generate final response

In [17]:
template = """
<|context|>
You are an Medical Assistant that Follows the instructions and generate the accurate response based on the query and the context provided,
Please be truthful and give direct answers.
</S>
<|user|>
{query}
</s>
<|assistant|>
"""


In [18]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [19]:
prompt = ChatPromptTemplate.from_template(template)

In [20]:
# Assuming the vector store is already created from your chunks and embeddings
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})


In [21]:
rag_chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
response = rag_chain.invoke(query)

llama_perf_context_print:        load time =   21393.35 ms
llama_perf_context_print: prompt eval time =   21393.15 ms /    74 tokens (  289.10 ms per token,     3.46 tokens per second)
llama_perf_context_print:        eval time =   32789.59 ms /    52 runs   (  630.57 ms per token,     1.59 tokens per second)
llama_perf_context_print:       total time =   54238.85 ms /   126 tokens
llama_perf_context_print:    graphs reused =         56


In [23]:
response

'The risk of heart attack increases with age, especially in men over 40 years old. Other factors that increase the risk of heart attack include smoking, high blood pressure, high cholesterol, family history of heart disease, and being overweight.'

In [ ]:
import sys
while True:
  user_input=input(f"Input query:")
  if user_input=="exit":
    print("exiting...")
    sys.exit()
  if user_input=="":
    continue
  response=rag_chain.invoke(user_input)
  print("Answer:",response)

Input query:heart attack


Llama.generate: 56 prefix-match hit, remaining 13 prompt tokens to eval
llama_perf_context_print:        load time =   21393.35 ms
llama_perf_context_print: prompt eval time =    3364.91 ms /    13 tokens (  258.84 ms per token,     3.86 tokens per second)
llama_perf_context_print:        eval time =   20359.95 ms /    32 runs   (  636.25 ms per token,     1.57 tokens per second)
llama_perf_context_print:       total time =   23758.62 ms /    45 tokens
llama_perf_context_print:    graphs reused =         30


Answer: I am sorry to hear that. A heart attack is a serious medical condition that requires immediate attention. Can you tell me more about the symptoms you are experiencing?
Input query:how to treat heart attack patient as a doctor


Llama.generate: 56 prefix-match hit, remaining 20 prompt tokens to eval
llama_perf_context_print:        load time =   21393.35 ms
llama_perf_context_print: prompt eval time =    7044.66 ms /    20 tokens (  352.23 ms per token,     2.84 tokens per second)
llama_perf_context_print:        eval time =   26048.83 ms /    41 runs   (  635.34 ms per token,     1.57 tokens per second)
llama_perf_context_print:       total time =   33137.78 ms /    61 tokens
llama_perf_context_print:    graphs reused =         39


Answer: As a doctor, the first step in treating a heart attack patient is to call 911 or your emergency response number. While waiting for the ambulance, you should follow these steps: 
